<a href="https://colab.research.google.com/github/PhoneNaingSoe/home/blob/main/Phone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
N_var = 12

In [2]:
import numpy as np
import pandas as pd

In [3]:
A_0 = np.identity(10) * 2 + np.eye(10, 10, k = -1) * -1 + np.eye(10, 10, k = 1) * -1

In [4]:
def construct_delta_A(N_var, eps):
    """
    Алгоритм получения матрицы delta_A
    Ввод: 
    N_var - номер варианта
    eps - параметр
    Вывод:
    delta_A
    """
    par = 10
    delta_A = []
    c = N_var * eps / (N_var + 1)
    for i in range(par):
        temp = []
        for j in range(par):
            temp.append(0 if i == j else c / (i + j))
        delta_A.append(temp)
    return np.array(delta_A)


In [5]:
eps = 0.1

In [6]:
def house(x):
    u_tilde = np.copy(x)
    u_tilde[0] += np.sign(x[0]) * np.linalg.norm(x)
    return u_tilde / np.linalg.norm(u_tilde)

def qr_householder(A):
    m, n = A.shape
    Q = np.identity(m)
    R = np.copy(A)
    for i in range(n):
        P_i         = np.identity(m) ## 1
        x           = R[i:, i] ## 2
        u           = house(x) ## 3
        P_streak    = np.identity(m - i) - 2 * np.outer(u, u) ## 4
        P_i[i:, i:] = np.copy(P_streak) ## 5
        R           = np.dot(P_i, R) ## 6
        Q           = Q @ P_i ## 7
    return Q[:, :n], R[:n, :] ## Обрезаем для матрицы Q (m-n) столбцов, для матрицы R - (m-n) строк

def qr_householder_effective(A):
    A_temp = np.copy(A)
    l = []
    m, n = A.shape
    for i in range(n):
        x                  = A_temp[i:, i] 
        u                  = house(x) 
        A_temp[i:m, i:n]  -= 2 * np.outer(u, np.matmul(u, A_temp[i:m, i:n]))
        A_temp[i + 1:m, i] = u[1:]
        l.append(u[0])
    return A_temp, l

Решение задачи

In [8]:
delta_A = construct_delta_A(N_var, 0.1)
A = A_0 + delta_A

amnt_columns = 1 # Количество удаляемых столбцов
A_hat = np.delete(A, [9 - i for i in range(amnt_columns)], axis = 1).round(2)
x_0 = np.random.rand(10 - amnt_columns) # Выберем случайный вектор x_0
b = np.dot(A_hat, x_0)
b_effective = np.copy(b)

Q, R = qr_householder(A_hat)
A_effective, u_first_elements = qr_householder_effective(A_hat)

for i in range(10 - amnt_columns):
    u = np.hstack((u_first_elements[i],A_effective[i + 1:, i]))
    gamma = -2 * np.inner(u, b_effective[i:])
    b_effective[i:] += gamma * u

assert ((Q @ R - A_hat) < 1e-6).all(), "QR разложение неправильное"

x = np.dot(np.linalg.inv(R) @ Q.T, b) # Решаем линейную задачу наименьших квадратов
x_effective = np.dot(np.linalg.inv(np.triu(A_effective[:-amnt_columns])), b_effective[:-amnt_columns])

value_estimation = np.linalg.norm(x - x_0) / np.linalg.norm(x_0) # Относительная погрешность
result = (f"Решение было получено. \n"
          f"x_0                       = {[round(i, 4) for i in x_0]} \n"
          f"Найдённое решение x       = {[round(i, 4) for i in x]} \n"
          f"Эффективное решение x     = {[round(i, 4) for i in x_effective]} \n"
          f"Относительная погрешность = {value_estimation:.4e}")

In [9]:
print(result)

Решение было получено. 
x_0                       = [0.3578, 0.1683, 0.6659, 0.7232, 0.7658, 0.7811, 0.5504, 0.9158, 0.448] 
Найдённое решение x       = [0.3578, 0.1683, 0.6659, 0.7232, 0.7658, 0.7811, 0.5504, 0.9158, 0.448] 
Эффективное решение x     = [0.3578, 0.1683, 0.6659, 0.7232, 0.7658, 0.7811, 0.5504, 0.9158, 0.448] 
Относительная погрешность = 6.3951e-16


In [10]:
eps = [1e-1, 1e-3, 1e-6]